<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/log_regression_scotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets 
from datasets import load_dataset
dataset = load_dataset('lex_glue','scotus')

In [ ]:
access="XXXX" # add token here 
model_name = 'danielsaggau/longformer_simcse_scotus'
train_batch_size = 6
max_seq_length = 4096
num_epochs = 1
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name, use_auth_token=access)

In [12]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 4096, 'do_lower_case': False}) with Transformer model: LongformerModel 
  (1): Pooling({'word_embedding_dimension': 512, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
train_loss = losses.MultipleNegativesRankingLoss(model)

In [25]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets, evaluation
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import torch
import random
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

In [ ]:
eval_df

In [6]:
train_df = dataset['train']
eval_df = dataset['validation']
text_col=train_df['text'] 
category_col=train_df['label']
x_eval = eval_df['text']
y_eval = eval_df['label']

In [ ]:
x_eval[1:10]

In [ ]:
st_model = 'danielsaggau/legal_long_bert'
orig_model = SentenceTransformer(st_model,use_auth_token=access)

In [ ]:
import pandas as pd 
pd.DataFrame(train_df)

In [8]:
train_samples =[]
for line in text_col:
        line = line.strip()
        if len(line) >= 10:
            train_samples.append(InputExample(texts=[line, line]))

In [ ]:
len(train_samples)

In [ ]:
X_train_noFT = orig_model.encode(text_col)
X_eval_noFT = orig_model.encode(x_eval)

In [16]:
sgd =  LogisticRegression(max_iter=5000)
sgd.fit(X_train_noFT, category_col)
y_pred_eval_sgd = sgd.predict(X_eval_noFT)

In [33]:
print('f1 micro No Fit', f1_score(y_eval, y_pred_eval_sgd, average="micro"))
print('f1 macro No Fit', f1_score(y_eval, y_pred_eval_sgd, average="macro"))

f1 micro No Fit 0.6928571428571428
f1 macro No Fit 0.5895014021719693


In [35]:
X_train = model.encode(text_col)
X_eval = model.encode(x_eval)

In [42]:
sgd =  LogisticRegression(max_iter=20000)
sgd.fit(X_train, category_col)
y_pred_eval_sgd = sgd.predict(X_eval)

In [46]:
print('f1 micro Fit', f1_score(y_eval, y_pred_eval_sgd, average="micro"))
print('f1 macro Fit', f1_score(y_eval, y_pred_eval_sgd, average="macro"))
print('Acc. SetFit', accuracy_score(y_eval, y_pred_eval_sgd))
print('Precision micro SetFit', precision_score(y_eval, y_pred_eval_sgd,average="micro"))
print('Precision macro SetFit', precision_score(y_eval, y_pred_eval_sgd,average="macro"))

f1 micro Fit 0.6957142857142857
f1 macro Fit 0.6085958331125504
Acc. SetFit 0.6957142857142857
Precision micro SetFit 0.6957142857142857
Precision macro SetFit 0.6487521171168334


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
